In [1]:
import tensorflow as tf

# Verify gpu installation 
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [46]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pandas as pd 
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import csv

with open('config.json') as config_file:
    config = json.load(config_file)

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

In [3]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("../models/research/object_detection/")
sys.path.append("../models/research/")

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(config['labels_path'], 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [6]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [21]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def size_from_coord(coordinates):
    return (coordinates[2]-coordinates[0])*(coordinates[3]-coordinates[1])

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = config['images']
dirs = []
for item in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if not os.path.isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, item)):
        dirs.append(PATH_TO_TEST_IMAGES_DIR + "/" + item)

for category_number, pathdir in enumerate(dirs):
    print(str(category_number) + ": " + pathdir)
    subdirs = [pathdir + "/" + f for f in os.listdir(pathdir) if os.path.isdir(os.path.join(pathdir, f))]
    for subdir in subdirs:
        TEST_IMAGE_PATHS = [f for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))]
        TEST_IMAGE_PATHS = [subdir + "/" + file for file in sorted(TEST_IMAGE_PATHS)]

        # Size, in inches, of the output images.
        IMAGE_SIZE = (12, 8)

        with detection_graph.as_default():
          with tf.Session(graph=detection_graph) as sess:
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            dd = []
            for idx, image_path in enumerate(TEST_IMAGE_PATHS):
              image = Image.open(image_path)
              # the array based representation of the image will be used later in order to prepare the
              # result image with boxes and labels on it.
              image_np = load_image_into_numpy_array(image)
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(image_np, axis=0)
              # Actual detection.

              (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
              dd.append((boxes, scores, classes, num))


        sizes = [size_from_coord(coord) for coord in boxes[0]]

        percentage_dictionary = {}

        for fn, frame in enumerate(dd):
            boxes, scores, classes, num = dd[fn]
            print("Frame nr: " + str(fn) + " (" + subdir + ")")
            for idx, score in enumerate(scores[0]):
                if score > 0.1:
                    class_nr = int(classes[0][idx])
                    item = category_index[class_nr]
                    #print(str(item) + " size: " + str(sizes[idx]))
                    if str(class_nr) in percentage_dictionary:
                        percentage_dictionary[str(class_nr)] += sizes[idx]
                    else: 
                        percentage_dictionary[str(class_nr)] = sizes[idx]
        #print(percentage_dictionary)

        for i in range(1,91):
            if str(i) not in percentage_dictionary:
                percentage_dictionary[str(i)] = 0

        int_docs_info = {int(k) : v for k, v in percentage_dictionary.items()}
        new_dict={}
        for key, value in sorted(int_docs_info.items()): # Note the () after items!
            new_dict[key]=value

        with open('movie_data.csv', 'a') as myfile:
            wr = csv.writer(myfile)
            valueslist = list(new_dict.values())
            valueslist.append(category_number)
            wr.writerow(valueslist)

In [98]:
dirs

['/home/khala501/data/ads/actuals/telco',
 '/home/khala501/data/ads/actuals/fashion',
 '/home/khala501/data/ads/actuals/fastfood',
 '/home/khala501/data/ads/actuals/fragrance',
 '/home/khala501/data/ads/actuals/cars',
 '/home/khala501/data/ads/actuals/alcoholic_beverages',
 '/home/khala501/data/ads/actuals/finance',
 '/home/khala501/data/ads/actuals/supermarket',
 '/home/khala501/data/ads/actuals/beverages',
 '/home/khala501/data/ads/actuals/vacation']

In [106]:
for category_number, pathdir in enumerate(dirs):
    print(str(category_number) + ": " + pathdir)
    subdirs = [pathdir + "/" + f for f in os.listdir(pathdir) if os.path.isdir(os.path.join(pathdir, f))]
    for subdir in subdirs:
        amountfiles = len([name for name in os.listdir(subdir)])
        print(str(amountfiles))

0: /home/khala501/data/ads/actuals/telco
71
51
51
131
1: /home/khala501/data/ads/actuals/fashion
41
61
79
61
2: /home/khala501/data/ads/actuals/fastfood
61
61
51
61
3: /home/khala501/data/ads/actuals/fragrance
61
61
91
65
4: /home/khala501/data/ads/actuals/cars
96
81
51
95
5: /home/khala501/data/ads/actuals/alcoholic_beverages
171
71
81
91
6: /home/khala501/data/ads/actuals/finance
71
55
71
81
7: /home/khala501/data/ads/actuals/supermarket
91
61
82
93
8: /home/khala501/data/ads/actuals/beverages
61
52
91
121
9: /home/khala501/data/ads/actuals/vacation
61
85
61
41
